In [74]:
import tensorflow as tf

In [75]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17686236815352820304
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4853137408
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17228742448382689561
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [76]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [197]:
from tensorflow.keras.datasets import mnist
from tensorflow import keras

In [198]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [199]:
x_train.shape

(60000, 28, 28)

In [200]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')/255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')/255

In [81]:
class CNNBlock(keras.layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super().__init__() #To run parent class tf.keras.layers.Layer
        self.Conv = keras.layers.Conv2D(out_channels, kernel_size, padding='same')
        self.BN = keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training=False): #Forward Method/Propagation
        x = self.Conv(input_tensor)
        x = self.BN(x, training=training)
        x = tf.nn.relu(x)
        return x

In [82]:
model = tf.keras.Sequential([
    CNNBlock(32),
    CNNBlock(64),
    CNNBlock(128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [83]:
model.compile(optimizer='adam',
              loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [56]:
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
938/938 [==============================] - 20s 20ms/step - loss: 0.4536 - accuracy: 0.9508
Epoch 2/3
938/938 [==============================] - 19s 20ms/step - loss: 0.0574 - accuracy: 0.9850
Epoch 3/3
938/938 [==============================] - 19s 20ms/step - loss: 0.0332 - accuracy: 0.98920s - loss: 0.033


In [57]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 5ms/step - loss: 0.0445 - accuracy: 0.9873


[0.04453292116522789, 0.9872999787330627]

In [84]:
class ResBlock(keras.layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__() #To run parent class tf.keras.layers.Layer
        self.cnn0 = CNNBlock(channels[0])
        self.cnn1 = CNNBlock(channels[1])
        self.cnn2 = CNNBlock(channels[2])
        self.pooling = tf.keras.layers.MaxPool2D()
        self.identity_mapping = keras.layers.Conv2D(channels[1], 1, padding='same')
    
    def call(self, input_tensor, training=False): #Forward Method/Propagation
        x = self.cnn0(input_tensor, training=training)
        x = self.cnn1(x, training=training)
        x = self.cnn2(x+self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)

In [85]:
class ResNetLike(keras.Model):
    def __init__(self, classes_num=10):
        super().__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = keras.layers.GlobalAvgPool2D()
        self.classifier = keras.layers.Dense(classes_num)
        
    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        return self.classifier(x)

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(x, self.call(x))

In [86]:
model = ResNetLike(classes_num=10)

In [87]:
model.compile(optimizer='adam',
              loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [62]:
model.fit(x_train, y_train, batch_size=64, epochs=3)

Epoch 1/3
938/938 [==============================] - 37s 38ms/step - loss: 0.0827 - accuracy: 0.9750
Epoch 2/3
938/938 [==============================] - 38s 40ms/step - loss: 0.0336 - accuracy: 0.9893
Epoch 3/3
938/938 [==============================] - 37s 40ms/step - loss: 0.0271 - accuracy: 0.9915


In [63]:
print(model.model().summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block_15 (ResBlock)      (None, 14, 14, 64)        28640     
_________________________________________________________________
res_block_16 (ResBlock)      (None, 7, 7, 256)         526976    
_________________________________________________________________
res_block_17 (ResBlock)      (None, 3, 3, 512)         1839744   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 2,400,490
Trainable params: 2,397,418
Non-trainable params: 3,072
_____________________________________________

In [64]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.0401 - accuracy: 0.9861


[0.040102772414684296, 0.9861000180244446]

In [109]:
import keras.layers as layer

class myModel(keras.Model):
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = layer.Dense(64)
        self.dense2 = layer.Dense(10)
        self.flat = layer.Flatten()
        
    def call(self, input_tensor, training=False):
        x = self.dense1(input_tensor)
        x = tf.nn.relu(x)
        x = self.flat(x)
        return self.dense2(x)

In [110]:
model = myModel()

In [111]:
model.compile(optimizer='adam',
              loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [112]:
model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3530 - accuracy: 0.8992
Epoch 2/2
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2923 - accuracy: 0.9176


In [113]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2787 - accuracy: 0.9216


[0.27867260575294495, 0.9215999841690063]

In [179]:
class Dense(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        
    def call(self, inputs):
        return tf.matmul(inputs, self.W) + self.b
    
    def build(self, input_shape):
        self.W = self.add_weight(name = 'W',
                                 shape=(input_shape[-1], self.units), 
                                 initializer="random_normal", 
                                 trainable=True)
        self.b = self.add_weight(name = 'b',shape=(self.units,), 
                                 initializer="zeros", 
                                 trainable=True)

In [208]:
import keras.layers as layer
class myReLU(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def call(self, input_tensor):
        return tf.math.maximum(0.0, input_tensor)
    
    
class myModel2(keras.Model):
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = Dense(128)
        self.dense2 = Dense(10)
        self.flat = layer.Flatten()
        self.relu = myReLU()
        
    def call(self, input_tensor, training=False):
        x = self.relu(self.dense1(input_tensor))
        x = self.flat(x)
        return self.dense2(x)

In [210]:
model = myModel2(num_classes=10)
model.compile(optimizer='adam',
              loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
)

In [211]:
model.fit(x_train, y_train, 
          batch_size=128, 
          epochs=5)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.3953 - accuracy: 0.8889
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.2861 - accuracy: 0.9197
Epoch 3/5
469/469 [==============================] - 6s 12ms/step - loss: 0.2757 - accuracy: 0.9231
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.2689 - accuracy: 0.9253
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.2651 - accuracy: 0.9253


In [212]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2712 - accuracy: 0.9252


[0.2711792588233948, 0.9251999855041504]

In [214]:
model.summary()

Model: "my_model2_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             multiple                  256       
_________________________________________________________________
dense_71 (Dense)             multiple                  1003530   
_________________________________________________________________
flatten_26 (Flatten)         multiple                  0         
_________________________________________________________________
my_re_lu_15 (myReLU)         multiple                  0         
Total params: 1,003,786
Trainable params: 1,003,786
Non-trainable params: 0
_________________________________________________________________
